In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import h5py as h5
filename = 'SNII.hdf5'
# Read H5 file
f = h5.File(filename, "r")
# Get and print list of datasets within the H5 file
datasetNames = [n for n in f.keys()]
for n in datasetNames:
    print(n)

Masses
Metallicities
Number_of_masses
Number_of_metallicities
Number_of_species
Reference
Species_names
Yield_names
Yields


In [66]:
Masses = f['Masses'][:]

In [67]:
Masses

array([   8.,    9.,   12.,   13.,   15.,   18.,   20.,   25.,   30.,
         40.,   60.,  100.,  120.], dtype=float32)

In [68]:
Metallicities = f['Metallicities'][:]

In [69]:
Metallicities

array([ 0.   ,  0.001,  0.004,  0.02 ], dtype=float32)

In [70]:
elements = f['Species_names'][:]

In [71]:
elements

array([b'Hydrogen', b'Helium', b'Carbon', b'Nitrogen', b'Oxygen', b'Neon',
       b'Magnesium', b'Silicon', b'Sulphur', b'Calcium', b'Iron'], dtype=object)

In [72]:
f['Yield_names'][:]

array([b'Z_0.00', b'Z_0.001', b'Z_0.004', b'Z_0.02'], dtype=object)

In [75]:
f['Yields/Z_0.001/Ejected_mass'][:]

array([  4.89106178,   5.70875597,   8.1308527 ,   8.69053936,
        10.25411034,  12.44572258,  14.89804363,  19.54817581,
        24.06858063,  31.77251053,  30.46581078,  51.58944321,  65.626091  ], dtype=float32)

In [76]:
f['Yields/Z_0.001/Yield'][:]

array([[ -7.79314339e-01,  -1.09488809e+00,  -2.09181118e+00,
         -1.88346457e+00,  -2.33148146e+00,  -3.21717787e+00,
         -5.03216553e+00,  -7.40000820e+00,  -9.68176746e+00,
         -1.35367889e+01,  -9.20366573e+00,  -1.87171211e+01,
         -2.64444199e+01],
       [  6.58688664e-01,   7.08719432e-01,   7.67138720e-01,
          9.03589010e-01,   1.53014314e+00,   2.08124375e+00,
          1.05877852e+00,   7.95940995e-01,   8.68318021e-01,
          1.19272101e+00,   4.31708527e+00,   7.75668573e+00,
          2.19261875e+01],
       [  2.45482605e-02,   5.08369617e-02,   9.85278487e-02,
          5.86317927e-02,   4.36407141e-02,   6.95605502e-02,
          6.62355274e-02,   1.16094023e-01,   5.43745980e-02,
          1.93091854e-02,   2.14126989e-01,   2.22935289e-01,
          2.12273151e-01],
       [  2.07313290e-03,   2.34984350e-03,   3.08062322e-03,
          2.57529248e-03,  -4.24361875e-04,  -4.47877275e-04,
          3.14351288e-03,   3.89899273e-04,  -1.993

In [77]:
f['Yields/Z_0.001/Total_Metals'][:]

array([  0.13855194,   0.40709192,   1.35447276,   1.01172721,
         0.83892071,   1.18154895,   4.02799034,   6.67571306,
         8.90166378,  12.46051788,   4.99824095,  11.14951515,   4.75875854], dtype=float32)

In [78]:
names = [n for n in f['Yields/Z_0.02'].keys()]

In [79]:
names

['Ejected_mass', 'Total_Metals', 'Yield']

## CC-SN YIELDS


In [267]:
indexing = {}
indexing['H'] = 'Hydrogen'
indexing['He'] = 'Helium'
indexing['C'] = 'Carbon'
indexing['N']= 'Nitrogen'
indexing['O'] = 'Oxygen'
indexing['Ne'] = 'Neon'
indexing['Mg'] = 'Magnesium'
indexing['Si'] = 'Silicon'
indexing['S'] = 'Sulphur' # Not used by TNG simulation
indexing['Ca'] = 'Calcium' # Not used by TNG simulation
indexing['Fe'] = 'Iron'

elements = list(indexing.keys())

table = {}

metallicities = list(f['Metallicities'].value)
masses = f['Masses'].value


for z_index,z in enumerate(metallicities):
    
    yield_subtable = {}
    
    z_name = f['Yield_names'].value[z_index].decode('utf-8')
    z_data = f['Yields/'+z_name+'/Yield']
     
    ejecta_mass = f['Yields/'+z_name+'/Ejected_mass'].value
    
    yield_subtable['Mass'] = masses
    remnants = masses-ejecta_mass
    yield_subtable['mass_in_remnants'] = np.divide(remnants,masses)
    for el in list(indexing.keys()):
        yield_subtable[el] = np.zeros(len(masses))
     
    summed_yields = np.zeros(len(masses))
        
    for m_index,mass in enumerate(masses):
        for el_index,el in enumerate(elements):
            el_yield = z_data[el_index][m_index]
            el_yield_fraction = el_yield/(mass-remnants[m_index])
            yield_subtable[el][m_index] = el_yield_fraction
            summed_yields[m_index]+=el_yield_fraction
         
    yield_subtable['unprocessed_mass_in_winds'] = 1.-summed_yields-yield_subtable['mass_in_remnants']
    
    
    table[z.astype(float)] = yield_subtable
    
    # Restructure table
    all_keys = ['Mass','mass_in_remnants','unprocessed_mass_in_winds']+elements

    list_of_arrays = [yield_subtable[key] for key in all_keys]
    restructure_subtable = np.core.records.fromarrays(list_of_arrays,names=all_keys)

    table[z] = restructure_subtable

        

In [268]:
table[metallicities[3]]

rec.array([ (   8.,  0.35942191,  0.64057808, -0.12350683,  0.1029522 ,  0.00032646,  0.00321482,  0.01195691,  0.00103193,  0.00114013,  0.00356276,  -6.92857648e-05,  -3.71730166e-05, -0.00057193),
 (   9.,  0.33912018,  0.66087982, -0.15501787,  0.10385986,  0.00233874,  0.00333841,  0.01976553,  0.00271937,  0.00102403,  0.00852475,   5.97119192e-03,   8.98001366e-04,  0.00657798),
 (  12.,  0.30720448,  0.69279552, -0.21484816,  0.08282538,  0.00722998,  0.00320906,  0.07619383,  0.01721282,  0.0040116 ,  0.00891201,   4.08016983e-03,   5.44621027e-04,  0.01062868),
 (  13.,  0.32168815,  0.67831188, -0.14169773,  0.09888977,  0.00231689,  0.00247367,  0.01482159,  0.00397462,  0.00409992,  0.00568473,   2.31677154e-03,   2.14743384e-04,  0.00690501),
 (  15.,  0.30451724,  0.69548277, -0.15058133,  0.12184691, -0.00104081,  0.0028229 ,  0.00606428,  0.00276856,  0.00486294,  0.0054797 ,   1.75553362e-03,   1.31738954e-04,  0.00588956),
 (  18.,  0.28380913,  0.71619087, -0.194821

In [19]:
f['Yields/Z_0.004/Yield'][1][5]/(12.588024)

0.15777491746376046

In [15]:
f['Yields/Z_0.004/Ejected_mass'][5]

12.588024

In [18]:
f['Masses'][5]

18.0